In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
from keras.utils import to_categorical

# Load the preprocessed CSV file
df = pd.read_csv('../data/lemmatized_and_misspelled_removed_SEFACED.csv')

# Split the data into features (lemmatized text) and target variable
X = df['lemmatized_tokens']
y = df['Class_Label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_encoded)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Define maximum sequence length
maxlen = 100

# Pad sequences to ensure uniform length
X_pad = pad_sequences(X_seq, maxlen=maxlen)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_one_hot, test_size=0.2, random_state=42)

# Define CNN model architecture
embedding_dim = 100
num_filters = 128
kernel_size = 5

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Predict and evaluate
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, y_pred))


Epoch 1/10
400/400 [==============================] - 32s 57ms/step - loss: 0.5805 - accuracy: 0.7723 - val_loss: 0.3106 - val_accuracy: 0.8725
Epoch 2/10
400/400 [==============================] - 24s 60ms/step - loss: 0.2316 - accuracy: 0.9130 - val_loss: 0.3103 - val_accuracy: 0.8763
Epoch 3/10
400/400 [==============================] - 25s 63ms/step - loss: 0.1607 - accuracy: 0.9394 - val_loss: 0.3354 - val_accuracy: 0.8797
Epoch 4/10
400/400 [==============================] - 22s 54ms/step - loss: 0.1179 - accuracy: 0.9550 - val_loss: 0.4045 - val_accuracy: 0.8719
Epoch 5/10
400/400 [==============================] - 23s 57ms/step - loss: 0.0909 - accuracy: 0.9649 - val_loss: 0.4531 - val_accuracy: 0.8716
Epoch 6/10
400/400 [==============================] - 21s 53ms/step - loss: 0.0745 - accuracy: 0.9704 - val_loss: 0.5338 - val_accuracy: 0.8678
Epoch 7/10
400/400 [==============================] - 22s 54ms/step - loss: 0.0648 - accuracy: 0.9748 - val_loss: 0.5832 - val_accuracy: